In [2]:
#### NAVER API

import requests
import mysql.connector
import time
from datetime import datetime

# 네이버 API 설정
NAVER_CLIENT_ID = 'c1M21TFbnPsLTH9OSif4'
NAVER_CLIENT_SECRET = 'futUGBEK8j'
NAVER_API_URL = 'https://openapi.naver.com/v1/search/news.json'

# MySQL 설정
MYSQL_HOST = 'localhost'
MYSQL_USER = 'root'
MYSQL_PASSWORD = '0000'
MYSQL_DATABASE = 'userdb'

# 특정 키워드 설정
KEYWORD = '삼성중공업'

def fetch_news_from_naver(keyword):
    # 헤더는 요청을 보낼 때 서버에 추가적인 정보를 전달하는 데 사용됩니다
    headers = {
        'X-Naver-Client-Id': NAVER_CLIENT_ID,
        'X-Naver-Client-Secret': NAVER_CLIENT_SECRET
    }
    # params는 URL에 추가될 쿼리 문자열을 포함하고 있으며, API에 전달할 검색 조건이나 옵션 등을 지정할 수 있습니다
    params = {
        'query': keyword,
        'display': 100,  # 최대 100개 뉴스 추출
        'start': 1,      # 시작 위치
        'sort': 'date'   # 날짜 기준으로 정렬
    }
    # requests.get 함수는 네이버 뉴스 API와 같은 웹 API에 요청을 보내고 응답을 받아옵니다
    response = requests.get(NAVER_API_URL, headers = headers, params = params)
    # response.raise_for_status()는 Python의 requests 라이브러리에서 사용되는 메서드로, HTTP 요청의 응답 상태 코드를 확인하고, 요청이 실패했을 경우 예외를 발생
    response.raise_for_status()
    return response.json()

def save_news_to_db(news_items):
    connection = mysql.connector.connect(
        host = MYSQL_HOST,
        user = MYSQL_USER,
        password = MYSQL_PASSWORD,
        database = MYSQL_DATABASE
    )
    cursor = connection.cursor()
    
    insert_query = """INSERT INTO news (title, description, link, pubDate) VALUES (%s, %s, %s, %s)"""
    check_query = """SELECT COUNT(*) FROM news WHERE link = %s"""


    
    # 네이버 API에서 받은 뉴스 기사 데이터베이스에 삽입
    for item in news_items:
        title = item['title'].replace('<b>', '').replace('</b>', '')  
        description = item['description'].replace('<b>', '').replace('</b>', '')
        link = item['link']
        pubDate = datetime.strptime(item['pubDate'], "%a, %d %b %Y %H:%M:%S %z")

        cursor.execute(check_query, (link,))
        count_result = cursor.fetchone()
        count = count_result[0] if count_result else 0

        if count == 0:
            document = (title, description, link, pubDate)
            cursor.execute(insert_query, document)
    
    connection.commit()
    cursor.close()
    connection.close()

def fetch_and_sort_news():
    connection = mysql.connector.connect(
        host = MYSQL_HOST,
        user = MYSQL_USER,
        password = MYSQL_PASSWORD,
        database = MYSQL_DATABASE
    )
    cursor = connection.cursor()
    
    # 뉴스 데이터를 pubDate 기준으로 정렬
    select_query = """SELECT title, description, link, pubDate FROM news ORDER BY pubDate DESC"""
    
    try:
        cursor.execute(select_query)
        news_items = cursor.fetchall()

    except mysql.connector.Error as err:
        print(f"Error: {err}")
    
    finally:
        cursor.close()
        connection.close()

    cursor.close()
    connection.close()

def main():
    try:
        while True:
            try:
                news_data = fetch_news_from_naver(KEYWORD)
                news_items = news_data.get('items', [])
                save_news_to_db(news_items)
                fetch_and_sort_news()
                print(f"{datetime.now()}: News data fetched and saved and sorted to database.")

            except Exception as e:
                print(f"Error during fetching or saving news: {e}")
            time.sleep(60)  # 1분 대기
    except KeyboardInterrupt:
        print("\nProgram terminated by user.")
    finally:
        print("Exiting program...")

if __name__ == "__main__":
    main()

2024-09-12 10:36:44.490531: News data fetched and saved and sorted to database.
2024-09-12 10:37:44.681111: News data fetched and saved and sorted to database.
2024-09-12 10:38:44.914861: News data fetched and saved and sorted to database.
2024-09-12 10:39:45.097702: News data fetched and saved and sorted to database.
2024-09-12 10:40:45.284804: News data fetched and saved and sorted to database.
2024-09-12 10:41:45.474951: News data fetched and saved and sorted to database.
2024-09-12 10:42:45.604452: News data fetched and saved and sorted to database.
2024-09-12 10:43:45.776741: News data fetched and saved and sorted to database.
2024-09-12 10:44:45.990628: News data fetched and saved and sorted to database.
2024-09-12 10:45:46.103237: News data fetched and saved and sorted to database.
2024-09-12 10:46:46.308324: News data fetched and saved and sorted to database.
2024-09-12 10:47:46.495844: News data fetched and saved and sorted to database.
2024-09-12 10:48:46.694450: News data fe